In [ ]:
import os
import logging
from datetime import datetime
from glob import glob
from typing import Dict, List, Optional, Union

# 외부 라이브러리 의존성 확인
try:
    from Bio.SeqUtils.ProtParam import ProteinAnalysis
except ImportError:
    raise ImportError("Biopython 라이브러리가 설치되지 않았습니다. 'pip install biopython'을 실행해주세요.")

class EPRMAnalyzer:
    """
    Effective Protein Recovery Mass (EPRM) Analyzer.

    단백질 서열의 물리화학적 특성(불안정성 지수, GRAVY)과 정제 키트의 효율을 기반으로
    실험 후 회수될 유효 단백질 농도를 예측하고 실험 가이드를 제공하는 클래스입니다.
    """

    def __init__(self, 
                 initial_conc_um: float = 10.0, 
                 initial_vol_ul: float = 90.0, 
                 final_vol_ul: float = 450.0):
        """
        초기화 메서드: 실험 조건 설정 및 결과 디렉토리 생성.

        Args:
            initial_conc_um (float): 초기 단백질 농도 (uM). 기본값 10.0.
            initial_vol_ul (float): 초기 부피 (ul). 기본값 90.0.
            final_vol_ul (float): 최종 희석/용출 부피 (ul). 기본값 450.0.
        """
        # 실험 파라미터
        self.c_start = initial_conc_um
        self.v_start = initial_vol_ul
        self.v_final = final_vol_ul
        self.eta_kit = 0.50  # NT-L021 등 키트 매뉴얼 기준 보수적 회수율 상수

        # 결과 저장소 설정 (Timestamp 기반 폴더링)
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.output_dir = f"EPRM_Results_{self.timestamp}"
        os.makedirs(self.output_dir, exist_ok=True)
        
        # 로깅 시스템 초기화
        self.log_path = os.path.join(self.output_dir, "eprm_analysis_detail.log")
        self._setup_logging()

    def _setup_logging(self) -> None:
        """
        로깅 핸들러 설정 (File + Stream).
        기존 핸들러를 제거하여 로그 중복 출력을 방지합니다.
        """
        logger = logging.getLogger()
        
        # 기존 핸들러 초기화
        if logger.hasHandlers():
            logger.handlers.clear()

        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - [%(levelname)s] - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S',
            handlers=[
                logging.FileHandler(self.log_path, encoding='utf-8'),
                logging.StreamHandler()
            ]
        )
        logging.info(f"Analysis Session Started. Output Directory: {self.output_dir}")

    def calculate_eprm(self, sequence: str) -> Dict[str, float]:
        """
        단백질 서열을 분석하여 예측 회수율과 유효 농도를 계산합니다.

        Logic:
            1. Biopython을 이용해 MW, Instability Index, GRAVY 계산.
            2. 안정성 계수(stab_factor)와 흡착 계수(ads_factor)를 통한 손실 보정.
            3. 키트 효율(eta_kit)과 결합하여 최종 농도 도출.

        Args:
            sequence (str): 아미노산 서열 문자열.

        Returns:
            Dict[str, float]: 분석 결과 (MW, 지수들, 보정 계수, 최종 농도 등).
        """
        # 1. 기초 물성 분석
        analysis = ProteinAnalysis(sequence)
        mw = analysis.molecular_weight() / 1000.0  # Da -> kDa
        instability_index = analysis.instability_index()
        gravy = analysis.gravy()
        
        # 2. 물리화학적 감쇄 로직 (Heuristic Damping Logic)
        # Instability가 20(보수적 기준)을 넘어가면 페널티 부여
        # 수식: 1.0 - (초과분 / 100)
        stab_factor = 1.0 - (max(0, instability_index - 20) / 100.0)
        
        # 소수성(Hydrophobicity)이 높을수록 튜브 흡착 등으로 인한 손실 가능성 증가
        # 수식: 1.0 - (|GRAVY| * 0.2)
        ads_factor = 1.0 - (abs(gravy) * 0.2)
        
        # 최종 단백질 고유 효율 계수 (0 이하가 되지 않도록 방어 코드 추가 권장)
        eta_prot = max(0.0, stab_factor * ads_factor)
        
        # 3. 농도 계산
        total_recovery_coeff = self.eta_kit * eta_prot
        
        # 이론적 최대 희석 농도 (단순 희석)
        c_theo_max = (self.c_start * self.v_start) / self.v_final
        
        # 보정된 유효 농도 (Effective Concentration)
        c_eff = c_theo_max * total_recovery_coeff
        
        return {
            "MW_kDa": mw,
            "Instability": instability_index,
            "GRAVY": gravy,
            "Total_Coeff": total_recovery_coeff,
            "C_Effective_uM": c_eff
        }

    def process_files(self) -> None:
        """
        현재 디렉토리의 .fasta 및 .txt 파일을 찾아 분석을 수행하고 로그를 기록합니다.
        """
        # 대상 파일 검색
        target_files = glob("*.fasta") + glob("*.txt")
        
        if not target_files:
            logging.error("No input files found (.fasta or .txt). Please place sequence files in the directory.")
            return

        logging.info(f"Found {len(target_files)} files to process.")

        for file_path in target_files:
            # 현재 실행 중인 파이썬 스크립트 파일은 제외
            if file_path == os.path.basename(__file__):
                continue
            
            logging.info(f"{'='*10} Processing File: {file_path} {'='*10}")
            
            try:
                with open(file_path, "r", encoding='utf-8') as f:
                    # FASTA 포맷 파싱 ('>' 기준으로 분리)
                    raw_content = f.read()
                    entries = raw_content.split('>')
                    
                    valid_entries = 0
                    for entry in entries:
                        if not entry.strip(): continue # 빈 항목 건너뛰기
                        
                        lines = entry.strip().split('\n')
                        header = lines[0].strip()
                        # 줄바꿈 제거 및 공백 제거, 대문자 변환
                        seq = "".join(lines[1:]).replace(" ", "").strip().upper()
                        
                        # 유효하지 않은 서열 건너뛰기 (예: 빈 서열)
                        if not seq: 
                            logging.warning(f"Skipping empty sequence: {header}")
                            continue
                        
                        # 분석 실행
                        res = self.calculate_eprm(seq)
                        valid_entries += 1
                        
                        # --- 결과 리포팅 ---
                        logging.info(f"[Target: {header}]")
                        logging.info(f"  • Molecular Weight: {res['MW_kDa']:.2f} kDa")
                        logging.info(f"  • Instability Index: {res['Instability']:.2f} (Threshold: 40, Conservative: 20)")
                        logging.info(f"  • GRAVY (Hydrophobicity): {res['GRAVY']:.3f}")
                        logging.info(f"  • Calculated Recovery Coeff: {res['Total_Coeff']:.4f} (Kit: {self.eta_kit})")
                        logging.info(f"  • >> Estimated Effective Conc: {res['C_Effective_uM']:.4f} uM")
                        
                        # 실험 가이드: 20nM 타겟 희석비 계산
                        if res['C_Effective_uM'] > 0:
                            dilution_factor = int(res['C_Effective_uM'] * 1000 / 20)
                            logging.info(f"  • [Exp Guide] For 20nM final conc, dilute 1:{dilution_factor}")
                        else:
                            logging.warning("  • [Exp Guide] Concentration too low for dilution calculation.")
                            
                    if valid_entries == 0:
                        logging.warning(f"No valid sequences found in {file_path}")

            except Exception as e:
                logging.error(f"Error processing file '{file_path}': {str(e)}")
        
        logging.info(f"{'='*40}")
        logging.info(f"All analysis completed. Check details in: {self.log_path}")

if __name__ == "__main__":
    # 인스턴스 생성 및 실행
    # 필요시 초기 파라미터 수정 가능: initial_conc_um=10.0, initial_vol_ul=90.0 등
    analyzer = EPRMAnalyzer()
    analyzer.process_files()